# tools for planning observations

In [3]:
__author__ = 'Pascal Louis <plouis35@gmail.com>'
__version__ = '1.0'
__keywords__ = ['astronomy', 'astropy', 'astroquery', 'astroplan', 'matplotlib']


# Targets

In [6]:
### Pises
#obs_loc = EarthLocation(lat=44.0*u.deg, lon=-3.5*u.deg, height=1300*u.m)

### CALC
obs_longitude = 1.5
obs_latitude = 48.0
obs_height = 50

targets = [
    '--',
### Exoplanets
    'WASP-14',
    'WASP-12',                      
    'HAT-P-22',    
    'HD 147506',        # HAT-P-2,                 
    'XO-1',                       
    'HD 189733',   
    'TOI-1141.01',
    'TOI 2017.01',
    'HD 135723',
    'HD 93148',   # Kelt-24 exo
    'GJ 3470',      # Exo neptune-tiède
    'NY Vir',               # EA+RPHS   | 13.30   |  14.22     |            |V |            |     |                  

### Eclipsing Binaries
                            #   Type      Max        Min I         Min II       Epoch        Year      Period           M-m/D  Spectrum 
    'HD 112409',            # BCEP:     |  5.14   |   5.16     |            |Hp|            |     |     0.324798       |     |B8V        
    'V0566 Oph',            # EW/KW     |  7.46   |   7.96     |   7.89     |V |41835.8617  |     |     0.40964569     |     |F4V   
    'AW Uma',               # EW/KW     |  7.75   |   8.48     |   8.43     |V |45765.7385  |     |     0.33363749     |     |F8Vp+F8Vp  
    'V0432 Aur',            # EA        |  7.98   |   8.40     |            |V |51571.4123  |     |     3.08175        |09   |G0    
    'WW Aur',               # EA        |  5.86   |   6.54     |   6.43     |V |52501.8139  |     |     2.52501936     |10  *|A4m+A5m    
    'V0781 Tau',            # EW/KW     |  8.9    |   9.3      |   9.3      |p |43874.954   |     |     0.3449100      |     |G0      
    'CN Lyn',               # EA        |  9.07   |   9.58     |            |Hp|            |     |                    |     |                 
    'BF Lyn',               # BY        |  7.72   |(  0.1     )|            |V |            |     |            
    'UV Leo',               # EA/DW     |  8.90   |   9.56     |   9.49     |V |38440.72633 |     |     0.60008478     |18   |G0V+G2V  
    'UW LMi',               # EA        |  8.44   |   8.67     |            |Hp|            |     |                    |     |     
    'DV Boo',               # EA        |  7.53   |   7.76     |   7.69     |V |48045.254   |     |     3.78264        |06   |A2   
    '44 Boo',               # EW        |  4.70   |   4.86     |   4.84     |V |52500.1807  |     |     0.2678190      |     |G2V+G2V   
    'V0916 Her',            # E:        |  8.03   |   8.38     |            |Hp|            |     |                    |     |     
    'HD 23642',             # EA        |  6.84   |   6.94     |   6.88     |V |52903.5981  |     |     2.461134       |     |A0V   
    'MR Del',               # EA        |  8.85   |   9.16     |            |Hp|            |     |         
    'HI Dra',               # RRC       |  9.02   |   9.20     |            |Hp|            |     |                
    'DD Mon',               # EB/KE     | 11.1    |  11.8      |  11.3      |p |30321.453   |     |     0.56801193     |     |B5     
    'V868 Mon',             # EB        |  8.9    |   9.5      |            |V |52681.731   |     |     0.63772        |   
    'MW Vir',               # EW:       |  7.00   |   7.04     |            |Hp|            |     |          
    'VW Cep',               # EW        |  7.37   |   7.77     |   7.73     |V |57625.370   |     |     0.2783085      |     |G5V+K0Ve    
    'HW Vir',               # EA/D      | 10.5    |(  0.90    )|            |V |            |     |                    |     |           

### Nova
    'T CrB',                # NR        |  2.0    |  10.8      |            |V |31860.      |1946 |(29000.           ) |     |M3III+pec(Nova)  
]

"""
### OHP 2022
targets = [
    'PN K  4-4', #                       281.04375 6.78194   PN K  4-4                       symb - no spectrum - V10.4
    'Iot Lyr', #                         286.82542 36.10028  Iot Lyr                         iot Lyr -- Be Star - V 5.253
    'CH Cyg', #                          291.13792 50.24139  CH Cyg                          symb - done
    'RR Lyr', #                          291.36625 42.78444  RR Lyr                          V* RR Lyr -- RR Lyrae Variable - R 7.6
    '12 Vul', #                          297.76708 22.61000  12 Vul                          12 Vul -- Be Star - V 4.96
    'SS 443', #                          297.95167 -5.81389  SS 443                          micro quasar - V 11.99
    'PN K  4-39', #                      298.27417 23.22667  PN K  4-39                      symb - no spectrum - V 8.8
    '25 Cyg', #                          299.98000 37.04278  25 Cyg                          25 Cyg -- Be Star - V 5.19
    'QR Vul', #                          303.81625 25.59194  QR Vul                          Double or Multiple Star - V 4.75
    'P Cyg', #                           304.44667 38.03306  P Cyg                           P Cyg -- Blue Supergiant - V 4.82
    '28 Vul', #                          309.63292 24.11611  28 Vul                          28 Vul -- Star - V 5.049
    'BW Vul', #                          313.59333 28.52194  BW Vul                          V* BW Vul -- beta Cep Variable - V 6.54
    'AG Peg', #                          327.75833 12.62556  AG Peg                          symb - done - V 8-9
    'Z And', #                           353.41667 48.81833  Z And                           symb - done -
    'R Aqr', #                           355.95625 -15.28444 R Aqr                           symb - done - V 6-10
    'EG And', #                          11.15500  40.67944  EG And                          symb - done - V 7.0
    'AX Per', #                          24.09458  54.25056  AX Per                          symb - done - V 10.5
    'Mira', #                            34.83667  -2.97694  HR681                           symb - done - V 2.0
    'UV Aur', #                          80.45375  32.51111  UV Aur                          symb - done - V 7.5
    'SS Lep', #                          91.24625  -16.48444 SS Lep                          symb - done - V 4.9
    'T CrB', #                           239.87583 25.92028  T CrB                           V* T CrB -- Symbiotic Star - V 10.247
]
"""

from IPython.display import display, IFrame
import ipywidgets as widgets
from easyplan import PlotPosition, moon_illumination
from datetime import date
from astropy.time import Time

output = widgets.Output()

def showTarget(name, date):
    with output:
        output.clear_output()
        ### moon phase
        print (f'moon phase: {moon_illumination(Time(str(date_selected.value), format="isot", scale="utc")):.0%}')

        ###
        obs_date = date
        target = name
        #obs_date = str(date_selected.value)
        #target = change['new']
        target = target_selected.value
    
        ### show position
        plan = PlotPosition(obs_latitude = obs_latitude, 
                            obs_longitude = obs_longitude, 
                            obs_height = obs_height,
                            obs_date = obs_date, 
                            name = target)
        display(plan)
        
        ### Show SIMBAD infos
        url = (f'https://simbad.cds.unistra.fr/simbad/sim-basic?Ident={target}&submit=SIMBAD+search')
        simbad_infos = IFrame(url, width='100%', height=700)
        display(simbad_infos)

date_selected = widgets.DatePicker(
    description='Date',
    value = date.today(),
    disabled=False
)
display (date_selected)

target_selected = widgets.Dropdown(options=targets, description='Target')
display(target_selected)

button = widgets.Button(
    description='Display',
    disabled=False,
    button_style='',
)

display(button)
def on_button_clicked(b):
    if target_selected.value != '--':
        showTarget(target_selected.value, str(date_selected.value))
    
button.on_click(on_button_clicked)
display(output)


DatePicker(value=datetime.date(2024, 4, 22), description='Date', step=1)

Dropdown(description='Target', options=('--', 'WASP-14', 'WASP-12', 'HAT-P-22', 'HD 147506', 'XO-1', 'HD 18973…

Button(description='Display', style=ButtonStyle())

Output()